In [ ]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from dotenv import load_dotenv

load_dotenv()

connection = mysql.connector.connect(
    host='54.207.19.250',
    user='gmvb_geilson',
    password='kK+(kB!Uh~$U',
    database='gmvb_crm'
)
cursor = connection.cursor()
caminho = 'C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/'
part_files = [caminho + "part -{}.csv".format(i) for i in range(15, 31)]

for part_file in part_files:
    file_path = os.path.join(".", part_file)  # Altere o diretório se necessário
    print("Lendo arquivo:", part_file)
    df_part = pd.read_csv(file_path, sep=';')
    
    df_part['TELEFONE'] = df_part['TELEFONE'].astype(object)

    df = pd.DataFrame(df_part)

    for indice in range(len(df)):
        origem = df['ORIGEM'][indice]
        telefone = df['TELEFONE'][indice]
        data_bloqueio = df['DATA_BLOQUEIO'][indice]
        data_arquivo = df['DATA_ARQUIVO'][indice]
        empresa = df['EMPRESA'][indice]
        origem_pedido = df['ORIGEMPEDIDO'][indice]
        data_inicio_bloqueio = df['DATA_INICIO_BLOQUEIO'][indice]

        consulta = "INSERT IGNORE INTO crm_nao_perturbe (origem, telefone, data_bloqueio, data_arquivo, empresa, OrigemPedido, data_inicio_bloqueio) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        valores = (origem, telefone, data_bloqueio, data_arquivo, empresa, origem_pedido, data_inicio_bloqueio)
        cursor.execute(consulta, valores)

    connection.commit()
    print(f"Dados no caminho {part_file} foram carregados com sucesso!\n")

connection.close()
print("Conexão Finalizada!")


Lendo arquivo: C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -15.csv
Dados no caminho C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -15.csv foram carregados com sucesso!

Lendo arquivo: C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -16.csv
Dados no caminho C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -16.csv foram carregados com sucesso!

Lendo arquivo: C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -17.csv
Dados no caminho C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -17.csv foram carregados com sucesso!

Lendo arquivo: C:/Users/geilson.maciel/Desktop/npMaster/Particionamentos/part -18.csv
